In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf
from keras.layers import Activation,Reshape,Dense,Embedding,Dropout
# from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
# from keras.applications.inception_v3 import InceptionV3,preprocess_input
# from keras.models import Model,load_model

from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,MaxPooling2D,Conv2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

In [9]:
train_data = (r"C:\Users\folas\Downloads\personal project\Deep Learning\emoji proj\train")
test_data = (r"C:\Users\folas\Downloads\personal project\Deep Learning\emoji project\validation")
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [10]:
# test_data_gen.flow_from_directory?
train_generator = train_data_gen.flow_from_directory(
    train_data,
    target_size = (48,48),
    batch_size= 64,
    color_mode = "grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.


In [11]:
test_generator = test_data_gen.flow_from_directory(test_data,
    target_size = (48,48),
    batch_size= 64,
    color_mode = "grayscale",
    class_mode="categorical"
)

Found 7066 images belonging to 7 classes.


In [12]:
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.05))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.05))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.05))


emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [13]:
# emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])
# es = EarlyStopping(min_delta = 0.1,monitor = 'accuracy',mode = 'min',patience = 6,restore_best_weights = True, verbose = 1)

In [21]:
# epochs = 60
# epochs_data =emotion_model.fit(train_generator, batch_size=123, epochs=epochs)
emotion_model_info = emotion_model.fit_generator(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=140,
    validation_data=test_generator,
    validation_steps=7178 // 64
)

Epoch 1/140
448/448 [==============================] - 140s 313ms/step - loss: 0.1267 - accuracy: 0.9618 - val_loss: 0.6596 - val_accuracy: 0.9104
Epoch 2/140
448/448 [==============================] - 124s 278ms/step - loss: 0.1312 - accuracy: 0.9582
Epoch 3/140
448/448 [==============================] - 119s 265ms/step - loss: 0.1209 - accuracy: 0.9642
Epoch 4/140
448/448 [==============================] - 130s 291ms/step - loss: 0.1398 - accuracy: 0.9579
Epoch 5/140
448/448 [==============================] - 134s 298ms/step - loss: 0.1239 - accuracy: 0.9614
Epoch 6/140
448/448 [==============================] - 130s 290ms/step - loss: 0.1369 - accuracy: 0.9573
Epoch 7/140
448/448 [==============================] - 120s 268ms/step - loss: 0.1268 - accuracy: 0.9604
Epoch 8/140
448/448 [==============================] - 2815s 6s/step - loss: 0.1290 - accuracy: 0.9601
Epoch 9/140
448/448 [==============================] - 114s 254ms/step - loss: 0.1201 - accuracy: 0.9636
Epoch 10/140
44

In [24]:
model = emotion_model.save_weights('emotion_model.h5')

In [22]:
model_accuracy = emotion_model.evaluate(test_generator)

111/111 [==============================] - 7s 63ms/step - loss: 0.6793 - accuracy: 0.9162


In [23]:
import cv2
cv2.ocl.setUseOpenCL(False)
emotion_dict ={0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

cap = cv2.VideoCapture(0)
while True:
    ret,frame = cap.read()
    if not ret:
        break
    box = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = box.detectMultiScale(gray_frame,scaleFactor=1.3,minNeighbors=5)
    for (x,y,w,h)in faces:
        frame = cv2.rectangle(frame,(x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_image = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_image)
        maxindex = int(np.argmax(emotion_prediction))
        frame = cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video',frame)
    key = cv2.waitKey(10)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             cap.release()
#             cv2.destroyAllWindows()
#             break